# MovieLens Recommender System Matrix Factorization 

In [0]:
#import libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

In [0]:
#load the data 
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
training = ratings.sample(70585)
testing = ratings.sample(30250)

In [0]:
#Construct a user-item table
mtx = np.array(training.pivot(index = 'userId', columns ='movieId', values = 'rating'))
mtx_df = pd.DataFrame(mtx)
mtx_fillmean = mtx_df.fillna(3.5)
matrix1 = np.array(mtx_fillmean)
matrix = matrix1 - np.asarray([(np.mean(matrix1, 1))]).T
matrix
UserItem = pd.DataFrame(matrix)
UserItem.head(6)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,8492,8493,8494,8495,8496,8497,8498,8499,8500,8501,8502,8503,8504,8505,8506,8507,8508,8509,8510,8511,8512,8513,8514,8515,8516,8517,8518,8519,8520,8521,8522,8523,8524,8525,8526,8527,8528,8529,8530,8531
0,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,...,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643,-0.016643
1,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,...,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820,-0.000820
2,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,-2.996425,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,...,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575,0.003575
3,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.500645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-1.500645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,...,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645,-0.000645
4,0.499531,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,0.499531,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,0.499531,0.499531,-0.000469,-0.500469,-0.000469,-0.000469,-0.000469,-0.000469,...,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.000469,-0.00046

Apporach 1: Making Predictions Using SGD model 

In [0]:
class MF():

  # Initializing the user-movie rating matrix, no. of latent features, alpha and beta.
  def __init__(self, R, K, alpha, beta, iterations):
      self.R = R
      self.num_users, self.num_items = R.shape
      self.K = K
      self.alpha = alpha
      self.beta = beta
      self.iterations = iterations

  # Initializing user-feature and movie-feature matrix 
  def train(self):
      self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
      self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

      # Initializing the bias terms
      self.b_u = np.zeros(self.num_users)
      self.b_i = np.zeros(self.num_items)
      self.b = np.mean(self.R[np.where(self.R != 0)])

      # List of training samples
      self.samples = [
      (i, j, self.R[i, j])
      for i in range(self.num_users)
      for j in range(self.num_items)
      if self.R[i, j] > 0
      ]

      # Stochastic gradient descent for given number of iterations
      training_process = []
      for i in range(self.iterations):
        np.random.shuffle(self.samples)
        self.sgd()
        mse = self.mse()
        training_process.append((i, mse))
      if (i+1) % 20 == 0:
        print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

  # Computing total mean squared error
  def mse(self):
      xs, ys = self.R.nonzero()
      predicted = self.full_matrix()
      error = 0
      for x, y in zip(xs, ys):
          error += pow(self.R[x, y] - predicted[x, y], 2)
      return np.sqrt(error)

  # Stochastic gradient descent to get optimized P and Q matrix
  def sgd(self):
      for i, j, r in self.samples:
          prediction = self.get_rating(i, j)
          e = (r - prediction)

          self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
          self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

          self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
          self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

  # Ratings for user i and moive j
  def get_rating(self, i, j):
      prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
      return prediction

  # Full user-movie rating matrix
  def full_matrix(self):
      return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

# Source Code Reference:
#https://github.com/thomasvn/Movie-Recommendation-System/blob/master/src/top_neighbors_per_dimension.py
#https://beckernick.github.io/matrix-factorization-recommender/
# https://gist.github.com/edwardyoon/e123853d1779975b5418ed43334aafc0

In [0]:
R= np.array(ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0))
mf = MF(R, K=100, alpha=0.001, beta=0.01, iterations=20)
training_process = mf.train()
print()
print("Predictions")
print(mf.full_matrix())
print()

Iteration: 20 ; error = 276.4621

Predictions
[[4.65345049 4.24277243 4.12999311 ... 4.23945939 4.24108868 4.25119196]
 [4.02208792 3.61008129 3.49938964 ... 3.61122295 3.61005751 3.62097915]
 [3.3223849  2.91062155 2.79723175 ... 2.90703367 2.9071818  2.92134738]
 ...
 [3.54373119 3.12862314 3.01823363 ... 3.1285079  3.13025723 3.14140584]
 [3.75289499 3.34086367 3.2304928  ... 3.33865106 3.3395075  3.35089751]
 [4.04815402 3.63392002 3.52329203 ... 3.63273783 3.63422996 3.64604104]]



Apporach 2: Making Predictions Using SVD Model


In [0]:
#Merge the two dataset together into one file (movieId)
mixmvrate = pd.merge(ratings, movies, on='movieId')
mixmvrate.head(3)
#drop missing values in movie titles
mixmvrate_clean = mixmvrate.dropna(axis = 0, subset = ['title'])
mixmvrate_clean.head(3)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [0]:
#Restructure the data, grouping them together by rate frequency and title. Rate frequency will the value in the matrix 
movie_ratingCount = (mixmvrate_clean.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'rate_frequency'})
     [['title', 'rate_frequency']]
    )
print(movie_ratingCount.head(3))
totalcountratetable = mixmvrate_clean.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
print(totalcountratetable.head(3))

                                     title  rate_frequency
0                               '71 (2014)               1
1  'Hellboy': The Seeds of Creation (2004)               1
2                   'Round Midnight (1986)               2
   userId  movieId  ...                                       genres  rate_frequency
0       1        1  ...  Adventure|Animation|Children|Comedy|Fantasy             215
1       5        1  ...  Adventure|Animation|Children|Comedy|Fantasy             215
2       7        1  ...  Adventure|Animation|Children|Comedy|Fantasy             215

[3 rows x 7 columns]


In [0]:
#Based on the new matrix, removing duplicate userId
userrate = totalcountratetable.drop_duplicates(['userId','title'])
userrate.head(3)

,userId,movieId,rating,timestamp,title,genres,rate_frequency
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215


In [0]:
#Using pandas pivot table to generate User-Item Matrix
titleUser = userrate.pivot(index = 'userId', columns = 'title', values = 'rating').fillna(0)
tu = np.array(titleUser)
titleUser
min_max_scaler = preprocessing.MinMaxScaler()
scaler = StandardScaler()
fit = scaler.fit_transform(titleUser)
df_normalized = pd.DataFrame(fit)
df_normalized.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9679,9680,9681,9682,9683,9684,9685,9686,9687,9688,9689,9690,9691,9692,9693,9694,9695,9696,9697,9698,9699,9700,9701,9702,9703,9704,9705,9706,9707,9708,9709,9710,9711,9712,9713,9714,9715,9716,9717,9718
0,-0.040522,-0.040522,-0.057354,-0.040522,-0.055636,-0.040522,-0.158111,-0.040522,-0.260796,-0.106728,-0.040522,-0.068971,-0.040522,-0.040522,-0.078527,-0.049159,-0.150327,-0.066967,-0.297815,-0.040522,-0.160279,-0.080528,-0.040522,-0.276719,-0.272028,-0.040522,-0.040522,-0.117688,-0.040522,-0.048364,-0.040522,-0.057354,-0.080705,-0.040522,-0.311156,-0.040522,-0.040522,-0.040522,-0.070302,-0.161482,...,-0.079771,-0.040522,-0.040522,-0.056776,-0.047697,-0.040522,-0.040522,-0.056237,-0.040522,-0.107501,-0.151281,-0.156175,-0.068418,-0.040522,-0.040522,-0.040522,-0.250495,-0.05324,-0.057354,-0.040522,-0.300332,-0.040522,-0.057354,-0.071965,-0.298684,-0.069378,-0.040522,-0.040522,-0.230241,-0.078803,-0.040522,-0.113246,-0.069302,-0.057354,-0.040522,-0.189406,-0.191107,-0.088701,5.845590,-0.040522
1,-0.040522,-0.040522,-0.057354,-0.040522,-0.055636,-0.040522,-0.158111,-0.040522,-0.260796,-0.106728,-0.040522,-0.068971,-0.040522,-0.040522,-0.078527,-0.049159,-0.150327,-0.066967,-0.297815,-0.040522,-0.160279,-0.080528,-0.040522,-0.276719,-0.272028,-0.040522,-0.040522,-0.117688,-0.040522,-0.048364,-0.040522,-0.057354,-0.080705,-0.040522,-0.311156,-0.040522,-0.040522,-0.040522,-0.070302,-0.161482,...,-0.079771,-0.040522,-0.040522,-0.056776,-0.047697,-0.040522,-0.040522,-0.056237,-0.040522,-0.107501,-0.151281,-0.156175,-0.068418,-0.040522,-0.040522,-0.040522,-0.250495,-0.05324,-0.057354,-0.040522,2.374160,-0.040522,-0.057354,-0.071965,-0.298684,-0.069378,-0.040522,-0.040522,-0.230241,-0.078803,-0.040522,-0.113246,-0.069302,-0.057354,-0.040522,-0.189406,-0.191107,-0.088701,-0.201999,-0.040522
2,-0.040522,-0.040522,-0.057354,-0.040522,-0.055636,-0.040522,-0.158111,-0.040522,-0.260796,-0.106728,-0.040522,-0.068971,-0.040522,-0.040522,-0.078527,-0.049159,-0.150327,-0.066967,-0.297815,-0.040522,-0.160279,-0.080528,-0.040522,-0.276719,-0.272028,-0.040522,-0.040522,-0.117688,-0.040522,-0.048364,-0.040522,-0.057354,-0.080705,-0.040522,-0.311156,-0.040522,-0.040522,-0.040522,-0.070302,-0.161482,...,-0.079771,-0.040522,-0.040522,-0.056776,-0.047697,-0.040522,-0.040522,-0.056237,-0.040522,-0.107501,-0.151281,-0.156175,-0.068418,-0.040522,-0.040522,-0.040522,-0.250495,-0.05324,-0.057354,-0.040522,-0.300332,-0.040522,-0.057354,-0.071965,-0.298684,-0.069378,-0.040522,-0.040522,-0.230241,-0.078803,-0.040522,-0.113246,-0.069302,-0.057354,-0.040522,-0.189406,-0.191107,-0.088701,-0.201999,-0.040522
3,-0.040522,-0.040522,-0.057354,-0.040522,-0.055636,-0.040522,-0.158111,-0.040522,-0.260796,-0.106728,-0.040522,-0.068971,-0.040522,-0.040522,-0.078527,-0.049159,-0.150327,-0.066967,-0.297815,-0.040522,-0.160279,-0.080528,-0.040522,-0.276719,-0.272028,-0.040522,-0.040522,-0.117688,-0.040522,-0.048364,-0.040522,-0.057354,-0.080705,-0.040522,3.701636,-0.040522,-0.040522,-0.040522,-0.070302,-0.161482,...,-0.079771,-0.040522,-0.040522,-0.056776,-0.047697,-0.040522,-0.040522,-0.056237,-0.040522,-0.107501,-0.151281,-0.156175,-0.068418,-0.040522,-0.040522,-0.040522,-0.250495,-0.05324,-0.057354,-0.040522,-0.300332,-0.040522,-0.057354,-0.071965,-0.298684,-0.069378,-0.040522,-0.040522,-0.230241,-0.078803,-0.040522,-0.113246,-0.069302,-0.057354,-0.040522,-0.189406,-0.191107,-0.088701,-0.201999,-0.040522
4,-0.040522,-0.040522,-0.057354,-0.040522,-0.055636,-0.040522,-0.158111,-0.040522,-0.260796,-0.106728,-0.040522,-0.068971,-0.040522,-0.040522,-0.078527,-0.049159,-0.150327,-0.066967,-0.297815,-0.040522,-0.160279,-0.080528,-0.040522,-0.276719,-0.272028,-0.040522,-0.040522,-0.117688,-0.040522,-0.048364,-0.040522,-0.057354,-0.080705,-0.040522,-0.311156,-0.040522,-0.040522,-0.040522,-0.070302,-0.161482,...,-0.079771,-0.040522,-0.040522,-0.056776,-0.047697,-0.040522,-0

In [0]:
#Using TruncatedSVD model to decompose the matrix. 
transT = df_normalized.values.T
transT.shape
SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(transT)
print(matrix.shape)
matrix

(9719, 12)


array([[ 4.01876951e+00, -3.88498141e+00,  1.86234037e+00, ...,
         5.74942658e-01,  1.02811646e+00,  5.03226539e-01],
       [ 8.93583358e-02,  3.37428724e-02, -1.45304961e-03, ...,
        -9.32391366e-02,  2.07692505e-01,  8.28230752e-02],
       [-2.58989349e-02,  9.26573701e-02, -1.84344310e-02, ...,
        -1.53843495e-01,  2.73083700e-01,  3.78375367e-01],
       ...,
       [ 3.71638071e+00, -2.51638761e+00,  1.70904963e+00, ...,
        -1.33703315e+00, -1.85100752e+00,  1.82723176e+00],
       [ 9.44466912e+00,  1.80684534e-01,  1.68196390e+00, ...,
        -1.39397699e+00,  6.39397934e-01,  6.04453581e-01],
       [ 2.05159996e-02,  7.38945249e-02, -2.21983473e-02, ...,
        -3.64891778e-01,  2.48107668e-01,  4.32103609e-02]])

In [0]:
#For movie * movie, I found the most common way is to use correlation as a similairty measure 
corr = np.corrcoef(matrix)
corr.shape
corr

array([[ 1.        , -0.07743097, -0.0965565 , ...,  0.64738218,
        -0.10049318, -0.16126592],
       [-0.07743097,  1.        ,  0.59713199, ...,  0.41478377,
         0.17438453,  0.55066404],
       [-0.0965565 ,  0.59713199,  1.        , ...,  0.01717963,
         0.05849915,  0.73249199],
       ...,
       [ 0.64738218,  0.41478377,  0.01717963, ...,  1.        ,
         0.12064561, -0.11584748],
       [-0.10049318,  0.17438453,  0.05849915, ...,  0.12064561,
         1.        ,  0.34180714],
       [-0.16126592,  0.55066404,  0.73249199, ..., -0.11584748,
         0.34180714,  1.        ]])

In [0]:
# List out the titles in the matrix for future reommender system 
moviet = titleUser.columns
moviet_list = list(moviet)

In [0]:
# User Interface : Input Movie Title, Return List of Remmendations 
# Recommendations for Toy Story 
mvlist = moviet_list.index("Toy Story (1995)")
corrmv = corr[mvlist]
output1 = pd.DataFrame(list(moviet[(corrmv >= 0.90)]), columns=['Recommended Movies'])[1:8]
output1
output1.reset_index(drop=True)

,Recommended Movies
0,Airplane! (1980)
1,Aliens (1986)
2,Animal House (1978)
3,Austin Powers: International Man of Mystery (1...
4,Austin Powers: The Spy Who Shagged Me (1999)
5,Back to the Future (1985)
6,Back to the Future Part II (1989)


In [0]:
# Recommendations for Avatar
mvlist = moviet_list.index("Avatar (2009)")
corrmv = corr[mvlist]
corrmv
output2 = pd.DataFrame(list(moviet[(corrmv >= 0.93)]), columns=['Recommended Movies'])[3:7]
output2.reset_index(drop=True)

,Recommended Movies
0,"Avengers, The (2012)"
1,Babylon A.D. (2008)
2,Balls Out: Gary the Tennis Coach (2009)
3,Beer League (2006)


In [0]:
# Recommendations for Ghost
mvlist = moviet_list.index("Ghost (1990)")
corrmv = corr[mvlist]
corrmv
output3 = pd.DataFrame(list(moviet[(corrmv >= 0.94)]), columns=['Recommended Movies'])[1:5]
output3.reset_index(drop=True)

,Recommended Movies
0,"Fly, The (1986)"
1,"Full Monty, The (1997)"
2,Ghost (1990)
3,"Good Morning, Vietnam (1987)"
